In [1]:
# Import Libraries

import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from sklearn.model_selection import KFold, train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

import plotly.offline as py
py.init_notebook_mode(connected=True)
import plotly.graph_objs as go
import plotly.tools as tls
import seaborn as sns
import pickle

In [2]:
# Function to reduce the DF size
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [3]:
# load the model from disk
model = pickle.load(open('random_forest_model.sav', 'rb'))

In [5]:
#Load test data
root = "D:/Kaggle/Energy Prediction/Data/"
df_test = pd.read_csv(f"{root}altered_test.csv")
sample = pd.read_csv(f"{root}sample_submission.csv")
df_test = reduce_mem_usage(df=df_test)

Mem. usage decreased to 596.49 Mb (79.2% reduction)


In [6]:
x_test = df_test.values
len(x_test)

41697600

In [7]:
#Prediction
predictions = model.predict(x_test)
sample['meter_reading'] = pd.Series(data=predictions) 

In [8]:
#Convert and save to submission format
from math import e
sample['meter_reading'] = e**sample['meter_reading'] - 1
sample.to_csv("baseline_submission.csv", index=False, float_format='%.4f')

In [9]:
sample.head(5)

,row_id,meter_reading
0,0,0.000000e+00
1,1,4.644712e+02
2,2,3.149382e+01
3,3,5.404613e+144
4,4,1.161057e+302
